[Pipelines](https://www.kaggle.com/code/alexisbcook/pipelines/tutorial)
> 전처리한 복잡한 모델을 배포 및 테스트하는데 중요한 기술

이번 장에서는 **Pipelines**를 통해 모델링 코드를 정리하는 방법을 배울 것이다. 
> [Pipeline 쉬운 설명](https://m.blog.naver.com/gdpresent/221730873049)  
> [Pipeline 자세한 설명 블로그](https://rfriend.tistory.com/729)  
> [Pipeline 활용](https://data-newbie.tistory.com/186)

# Introduction
**Pipelines**는 데이터 전처리 및 모델링을 정리할 수 있는 간단한 방법이다. 특히, 파이프라인은 전처리 및 모델링 단게를 번들하므로 전체 번들을 단일 단계처럼 사용할 수 있다. 

많은 데이터 과학자들은 파이프라인 없이 모델을 해킹하지만 파이프라인에는 몇가지 중요한 이점이 있으며, 여기에는 아래 내용이 포함된다. 

1. **Cleaner Code**(코드 정리) : 전처리의 각 단계에서 데이터를 설명하는 것은 혼란스러울 수 있다. 파이프 라인을 사용하면 각 단계에서 교육 및 검증 데이터를 수동으로 추적할 필요가 없어진다. 
2. **Fewer Bugs**(버그 감소) : 단계를 잘못 적용하거나 전처리 단계를 잊어버릴 가능성이 더 작아진다. 
3. **Easier to Productionzie**(생산성 용이) : 프로토타입에서 대규모로 배포할 수 있는 모델로 전환하는 것은 어려울 수 있다. 여기서는 언급하지 않겠지만, 파이프라인은 생산 용이성에 도움이 될 수 있다.
4. **More Options for Model Validation**(모델 유효성 검사를 위한 추가 옵션): 교차 검증을 다루는 자습서가 아래에 예시로 나와 있다.

# Example
지난 튜토리얼처럼 멜버른 집값 데이터를 사용하도록 한다. 

데이터 불러오기 과정은 넘기고 훈련/검증 데이터가 이미 있다고 가정한다. (X_train,X_valid,y_train,y_valid)

In [74]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터 불러오기
data=pd.read_csv('./data/melb_data.csv')

# 목표/예측 변수 분리
y=data.Price
X=data.drop(['Price'],axis=1)

# 데이터를 훈련 및 검증 데이터로 분리
X_train_full, X_valid_full, y_train, y_valid=train_test_split(X,y,train_size=.8, test_size=.2, random_state=0)

# 'Cardinality'는 열의 고유한 숫자값을 의미한다.
# Cardinality가 상대적으로 낮은 범주형 열 선택(편의를 위한 임의 과정)
cat_cols=[cname for cname in X_train_full.columns if X_train_full[cname].nunique()<10 and X_train_full[cname].dtype=='object']

# 수치형 변수 선택
num_cols=[cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64','float64']]

# 선택된 열
my_cols=cat_cols+num_cols
X_train=X_train_full[my_cols].copy()
X_valid=X_valid_full[my_cols].copy()

**head()**를 통해 training data를 살펴보도록 하자. 데이터에는 범주형 데이터와 결측값이 있는 열이 모두 포함된다. 파이프사인을 사용하면 두 가지 모두를 쉽게 처리할 수 있다. 

In [59]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


여기서는 세 단계로 파이프라인을 구성한다. 

## Step1. Define Preprocessing Steps(전처리 단계 정의)
파이프라인에서 전처리 및 모델링 단계를 함께 묶는 방법과 유사하게 **ColumnTransformer** 클래스를 사용해 서로 다른 전처리 단계를 함께 묶는다. 

**ColumnsTransformer 역할**
* 결측값을 **수치 데이터**에 귀속시킨다. 
* 결측값을 귀속시키고 **범주형 데이터**에 원한인코딩을 적용한다. 

In [64]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# 수치형 데이터 전처리
numerical_transformer=SimpleImputer(strategy='constant')

# 범주형 데이터 전처리
categorical_transformer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])

# 수치/범주형 데이터 전처리를 위한 번들
preprocessor = ColumnTransformer(
transformers=[
    ('num', numerical_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
])

In [66]:
preprocessor

ColumnTransformer(transformers=[('num', SimpleImputer(strategy='constant'),
                                 ['Rooms', 'Distance', 'Postcode', 'Bedroom2',
                                  'Bathroom', 'Car', 'Landsize', 'BuildingArea',
                                  'YearBuilt', 'Lattitude', 'Longtitude',
                                  'Propertycount']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Type', 'Method', 'Regionname'])])

## Step2. Define the Model(모델 정의)
다음, 우리는 친숙한 **RandomForestRegressor** 클래스로 RandomForest 모델을 정의한다. 

In [54]:
from sklearn.ensemble import RandomForestRegressor

model=RandomForestRegressor(n_estimators=100, random_state=0)

## Step3. Create and Evaluate the Pipeline(파이프라인 생성 및 평가)
마지막으로, 우리는 파이프라인 클래스를 사용해 전처리 및 모델링 단계를 번들로 하는 파이프라인을 정의한다. 주의해야할 몇가지 중요한 사항이 있다. 

* 파이프라인을 사용해 훈련 train data를 전처리하고 모델을 단일 코드 라인에 맞춘다. 
    - 파이프라인이 없다면 대치, 원핫 인코딩 및 모델 training을 별도로 수행해야 한다. 
    - 숫자/범주형 변수를 모두 처리해야 할 경우 특히 더 복잡해진다. 
   
   
* 파이프라인을 사용해 **X_valid()** 전처리 되지 않은 변수를 **predict()**에 넘겨주고, 파이프라인이 예측을 생성하기 전에 변수 전처리를 진행한다. 
    - 파이프라인이 없다면, 예측전에 검증 데이터를 전처리해야 한다는 것을 기억해야 한다. 

In [78]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn import set_config

# 파이프라인의 모델링 및 전처리 코드 번들
my_pipeline=Pipeline(steps=[('preprocess',preprocessor),
                           ('model',model)
                           ])

# 머신러닝 워크플로우 파이프라인 다이어그램으로 시각화
set_config(display='diagram')

# train 데이터의 훈련 및 모델 적합.
my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['Rooms', 'Distance',
                                                   'Postcode', 'Bedroom2',
                                                   'Bathroom', 'Car',
                                                   'Landsize', 'BuildingArea',
                                                   'YearBuilt', 'Lattitude',
                                                   'Longtitude',
                                                   'Propertycount']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Type', 'Method',
                                                   'Regionname'])])),
                ('model', RandomForestRegressor(random_state=0))])

In [79]:
# 검증 데이터 전처리 및 예측
preds=my_pipeline.predict(X_valid)

# 모델 평가
score=mae(y_valid, preds)
print('MAE : ',score)

MAE :  160679.18917034855


In [75]:
print(my_pipeline.score(X_train, y_train))

0.9711809490692984


# Conculsion
파이프라인은 머신러닝 코드를 정리하고 오류를 방지하는데 유용하며, 특히 정교한 데이터 전처리가 가능한 워크플로우에 유용하다. 
